In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [22]:
foldername= "/content/drive/My Drive/nlpproject/"

In [23]:
import numpy as np 
import pandas as pd 
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments, Trainer
from datasets import load_metric
import datetime
from torch import nn
from transformers import AutoConfig
from transformers import AutoModel
from tqdm.auto import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold
from torch.nn.modules.loss import CrossEntropyLoss

In [24]:
class CFG:
    str_now = datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
    basic_lr=1e-3
    train = True
    debug = False
    offline = False
    models_path = "bert-base-uncased"
    epochs = 50000
    save_all_models = False
    apex = True
    print_freq = 20
    num_workers = 4
    model = "bert-base-uncased"
    loss_func = 'SmoothL1'
    scheduler = 'cosine'
    batch_scheduler = True
    num_cycles = 0.5
    num_warmup_steps = 0
    encoder_lr = 2e-5
    decoder_lr = 2e-5
    min_lr = 1e-6
    llrd = True
    layerwise_lr = 5e-5
    layerwise_lr_decay = 0.9
    layerwise_weight_decay = 0.01
    layerwise_adam_epsilon = 1e-6
    layerwise_use_bertadam = False
    #pooling
    pooling = 'mean' # mean, max, min, attention, weightedlayer
    layer_start = 4
    #init_weight
    init_weight = 'normal' # normal, xavier_uniform, xavier_normal, kaiming_uniform, kaiming_normal, orthogonal
    #re-init
    reinit = True
    reinit_n = 1
    #adversarial
    fgm = False
    awp = False
    adv_lr = 1
    adv_eps = 0.2
    unscale = False
    eps = 1e-6
    betas = (0.9, 0.999)
    max_len = 512
    weight_decay = 0.01
    gradient_accumulation_steps = 1
    max_grad_norm = 1000
    target_cols = ['EI', 'SN', 'TF', 'JP']
    seed = 42
    cv_seed = 42
    n_fold = 4
    trn_fold = list(range(n_fold))
    batch_size = 10000
    n_targets = 4
    gpu_id = 0
    device = f'cuda:{gpu_id}'
    hidden_size = 768
    init_weight="normal"
cfg=CFG()

In [25]:
from transformers import BertTokenizer
tokenizer=BertTokenizer.from_pretrained(cfg.model)

In [26]:
#path=foldername+"dataset2(sep).csv"
#print(dataset[0])
data_collator=DataCollatorWithPadding(tokenizer=tokenizer)
def getdl(ds):
    total_len=len(ds)
    train_len=int(len(ds)*0.9)
    val_len=int((total_len-train_len)/2)
    test_len=total_len-train_len-val_len
    [train_ds, val_ds, test_ds]=torch.utils.data.random_split(ds, [train_len, val_len, test_len])
    #return (training dataloader, validation dataloader, test dataloader)
    return len(train_ds), len(val_ds), len(test_ds), DataLoader(train_ds, batch_size=cfg.batch_size, shuffle=False, collate_fn=data_collator), DataLoader(val_ds, batch_size=cfg.batch_size, shuffle=False, collate_fn=data_collator), DataLoader(test_ds, batch_size=cfg.batch_size, shuffle=False, collate_fn=data_collator)
    #return DataLoader(ds, batch_size=cfg.batch_size, shuffle=False, collate_fn=data_collator)

In [27]:
#On GPU
input_ids=torch.load(foldername+'EI_Extraced_feature_tensors_50*8000_lasthidden.pt')
labels=torch.load(foldername+'EI_Extraced_label_tensors_50*8000_lasthidden.pt')
#On CPU
#input_ids=torch.load(foldername+'Extraced_feature_tensors.pt', map_location=torch.device('cpu'))
#labels=torch.load(foldername+'Extraced_label_tensors.pt', map_location=torch.device('cpu'))

In [28]:
num_one=torch.count_nonzero(labels, dim=0)
num_zero=len(labels)-num_one

print(num_zero)
print(num_one)
LABEL_RATIO=torch.tensor([num_one/num_one, num_zero/num_one])
LABEL_RATIO

tensor(303868)
tensor(92655)


tensor([1.0000, 3.2796])

In [29]:
from torch.utils.data import TensorDataset
a=1
b=0
class Extracted_ds(Dataset):
    def __init__(self, input_ids, labels):
        self.features=input_ids
        self.labels=labels
    def __len__(self):
        return (len(self.labels))
    def __getitem__(self, index):
        input_id=self.features[index]
        if self.labels[index]==1:
          label=a
        else:
          label=b
        return {"input_ids": input_id, "labels":label}

In [30]:
ds=Extracted_ds(input_ids, labels)

In [31]:
len_train, len_val, len_test, train_dl_EI,val_dl_EI, test_dl_EI=getdl(ds)

In [32]:
class myModel(nn.Module):
    def __init__(self, CFG, dropoutrate):
        super().__init__()
        self.CFG = CFG
        self.relu = nn.ReLU()
        self.t = nn.Tanh()

        self.fc1 = nn.Linear(768, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 64)
        self.fc4 = nn.Linear(64, 16)
        #self.fc5 = nn.Linear(512, 1)
        #self.fc6 = nn.Linear(256, 128)
        #self.fc7 = nn.Linear(128, 64)
        #self.fc8 = nn.Linear(64, 32)
        #self.fc9 = nn.Linear(32, 16)
        #self.fc10 = nn.Linear(16, 2)
        self.fc_final = nn.Linear(16, 2)

        self.dropout = nn.Dropout(p=dropoutrate, inplace=False)
    
    def forward(self, input_ids):
        output = self.dropout(input_ids)

        output = self.fc1(output)
        output = self.relu(output)
        output = self.dropout(output)

        output = self.fc2(output)
        output = self.relu(output)
        output = self.dropout(output)

        output = self.fc3(output)
        output = self.relu(output)
        output = self.dropout(output)

        output = self.fc4(output)
        output = self.relu(output)
        output = self.dropout(output)

        #output = self.fc5(output)
        #output = self.t(output)
        #output = self.dropout(output)

        #output = self.fc6(output)
        #output = self.t(output)
        #output = self.dropout(output)

        #output = self.fc7(output)
        #output = self.t(output)
        #output = self.dropout(output)

        #output = self.fc8(output)
        #output = self.t(output)
        #output = self.dropout(output)

        #output = self.fc9(output)
        #output = self.t(output)
        #output = self.dropout(output)

        output = self.fc_final(output)
        #output = self.sig(output)

        #output = torch.squeeze(output)
        return output #2 number between 0 and 1

In [33]:
def evaluate(labels, outputs):
  answers=(torch.argmax(outputs, dim=1))
  allcorrect=torch.sum(answers==labels)
  return allcorrect

In [34]:
from torch.optim import lr_scheduler
from torch import nn
from torch import optim
from torch.optim import Adam
from tqdm.notebook import tqdm
def train(train_ds, eval_ds, model, epochs, cfg, type, lr, loss=None):
    if torch.cuda.is_available():  
        dev = "cuda:0" 
    else:  
        dev = "cpu" 
    device = torch.device(dev)
    model = model.to(device)

    #weights=torch.tensor([1., 3.]).cuda()
    #criterion = nn.MSELoss()
    criterion = CrossEntropyLoss(weight=LABEL_RATIO)

    criterion.to(device)
    #criterion = loss
    
    optimizer = Adam(model.parameters(), lr=lr)
    #optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer,T_0=500, eta_min=1e-15)
    scheduler= optim.lr_scheduler.ExponentialLR(optimizer, 0.99, last_epoch=- 1, verbose=False)
        
    totalevalloss=0
    totalcorrect=0
    totaldata=0
    with torch.no_grad():
        model.eval()
        for batch in eval_ds:
            batch.to(device)
            blabels=batch["labels"]
            outputs=model(input_ids=batch["input_ids"])
            eloss=criterion(outputs, blabels).item()
            totalevalloss+=eloss
            totalcorrect+=evaluate(blabels, outputs)
            totaldata+=len(blabels)
    totalcorrect_rate=(totalcorrect/(totaldata))
    print("probability that our prediction of ", type, " is correct: ", totalcorrect_rate)
    #print(f'Initial Val Loss: {totalevalloss / len(eval_ds): .3f} ' ) 
    print(f'Initial Val Loss: {totalevalloss / len(eval_ds): .3f} | current lr: {scheduler.get_last_lr()}' ) 
    
    for e in range(epochs):
        totaltrainloss=0
        totaltraincorrect=0
        totaltraindata=0
        for i,batch in enumerate(train_ds):
            model.train()
            optimizer.zero_grad()
            batch.to(device)
            labels=batch["labels"]
            outputs=model(input_ids=batch["input_ids"])
            bloss=criterion(outputs, labels)
            bloss.backward()
            optimizer.step()
            totaltrainloss+=bloss.item()
            totaltraincorrect+=evaluate(labels, outputs)
            totaltraindata+=len(labels)
        scheduler.step()
        totalevalloss=0
        totalcorrect=0
        totaldata=0
        with torch.no_grad():
            model.eval()
            for batch in eval_ds:
                batch.to(device)
                blabels=batch["labels"]
                outputs=model(input_ids=batch["input_ids"])
                eloss=criterion(outputs, blabels).item()
                totalevalloss+=eloss
                totalcorrect+=evaluate(blabels, outputs)
                totaldata+=len(blabels)
        totalcorrect_rate=(totalcorrect/(totaldata))
        totaltraincorrect_rate = (totaltraincorrect/(totaltraindata))
        print("probability that our prediction of ", type, " is correct: ", totalcorrect_rate)
        print("probability that our prediction of ", type, " is correct in training dataset: ", totaltraincorrect_rate)
        #print(f'Epoch: {e+ 1} | Train Loss: {totaltrainloss / len(train_ds): .8f} | Val Loss: {totalevalloss / len(eval_ds): .3f}' ) 
        print(f'Epoch: {e+ 1} | Train Loss: {totaltrainloss / len(train_ds): .8f} | Val Loss: {totalevalloss / len(eval_ds): .8f} | current lr: {scheduler.get_last_lr()}' ) 

In [35]:
model=myModel(cfg, 0.2)
criterion = CrossEntropyLoss()
with torch.no_grad():
  device="cpu"
  for batch in train_dl_EI:
    blabels=batch["labels"]
    blabels.to(device)
    input_ids=batch["input_ids"]
    input_ids.to(device)
    outputs=model(input_ids=input_ids)
    print(outputs)
    print(blabels)
    eloss=criterion(outputs, blabels).item()
    break

tensor([[0.0997, 0.2187],
        [0.1121, 0.2092],
        [0.0787, 0.1763],
        ...,
        [0.0954, 0.2135],
        [0.0791, 0.1932],
        [0.0814, 0.2279]])
tensor([0, 0, 0,  ..., 0, 0, 1])


In [36]:
model=myModel(cfg, 0.15)
train(train_dl_EI, val_dl_EI, model, epochs=cfg.epochs, cfg=cfg, type="EI", lr=1e-2, loss=None)

probability that our prediction of  EI  is correct:  tensor(0.2323, device='cuda:0')
Initial Val Loss:  0.699 | current lr: [0.01]
probability that our prediction of  EI  is correct:  tensor(0.4622, device='cuda:0')
probability that our prediction of  EI  is correct in training dataset:  tensor(0.4433, device='cuda:0')
Epoch: 1 | Train Loss:  0.69920418 | Val Loss:  0.68740672 | current lr: [0.0099]
probability that our prediction of  EI  is correct:  tensor(0.5208, device='cuda:0')
probability that our prediction of  EI  is correct in training dataset:  tensor(0.5213, device='cuda:0')
Epoch: 2 | Train Loss:  0.68498939 | Val Loss:  0.68165344 | current lr: [0.009801]
probability that our prediction of  EI  is correct:  tensor(0.5519, device='cuda:0')
probability that our prediction of  EI  is correct in training dataset:  tensor(0.5577, device='cuda:0')
Epoch: 3 | Train Loss:  0.68201640 | Val Loss:  0.68004867 | current lr: [0.00970299]
probability that our prediction of  EI  is corr

KeyboardInterrupt: ignored